In [25]:
import pandas as pd
import numpy as np

wallmart = pd.read_csv("full_table.csv")

In [108]:
def remove_space(s):
    t = s.replace(" ","").replace("+","")
    return [t[i:i+2] for i in xrange(0, len(t), 2)]

seperated_codes = wallmart['codesum'].map(remove_space)

def grab_plus(s):
    t = s.replace(" ","")
    return [t[i-2:i+1] for i in xrange(0, len(t), 2) if t[i] == '+']

codes_plus = wallmart['codesum'].map(grab_plus)

big_list = []

def master_list(s):
    for i in s:
        if i not in big_list:
            big_list.append(i)

seperated_codes.map(master_list)            
codes_plus.map(master_list)

big_list

['RA',
 'FZ',
 'FG',
 'BR',
 'UP',
 'MI',
 'SN',
 'HZ',
 'VC',
 'TS',
 'DZ',
 'BL',
 'BC',
 'DU',
 'SQ',
 'PL',
 'FU',
 'GR',
 'GS',
 'SG',
 'PR',
 'FG+']

In [112]:
for i in big_list:
    wallmart[i] = wallmart['codesum'].str.contains(i).astype(int)

wallmart = wallmart.drop('codesum', 1)

In [26]:
def get_holidays(fpath):
    f = open(fpath)
    lines = f.readlines()
    lines = [line.split(" ")[:3] for line in lines]
    lines = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines = pd.to_datetime(lines)
    return pd.DataFrame({"date2":lines})

holidays = get_holidays("holidays.txt")

def is_spring(date):
    date = date.timetuple().tm_yday
    if date in range(80, 172):
        s = 1  # spring
    else:
        s = 0
    return s

def is_summer(date):
    date = date.timetuple().tm_yday
    if date in range(172, 264):
        s = 1  # summer
    else:
        s = 0
    return s

def is_fall(date):
    date = date.timetuple().tm_yday
    if date in range(264, 355):
        s = 1  # fall
    else:
        s = 0
    return s

wallmart['date'] = pd.to_datetime(wallmart['date'])
wallmart['weekday'] = wallmart['date'].dt.weekday
wallmart['is_weekend'] = wallmart['weekday'].isin([5,6]).astype(int)
wallmart['is_holiday'] = wallmart['date'].isin(holidays.date2).astype(int)
wallmart['is_holiday_weekday'] = (wallmart['is_holiday'] & wallmart['is_weekend'] == 0).astype(int)
wallmart['is_holiday_weekend'] = (wallmart['is_holiday'] & wallmart['is_weekend']).astype(int)

wallmart['spring'] =  wallmart['date'].map(is_spring)
wallmart['summer'] =  wallmart['date'].map(is_summer)
wallmart['fall'] =  wallmart['date'].map(is_fall)

wallmart = wallmart.drop('weekday', 1)